# Đồ án môn Lập trình song song ứng dụng
**Giảng viên: Thầy Trần Trung Kiên**

## Thông tin nhóm
### STT: Nhóm 1

### Thành viên:
1. 1712892 - Nguyễn Thị Tuyết
2. 1712284 - Hoàng Gia Bảo
3. 1712685 - Trần Huy Phượng

## 1. Mô tả ứng dụng

Tên ứng dụng: **Phát hiện cộng đồng trong mạng xã hội bằng thuật toán Girvan Newman**

* Input: Đồ thị vô hướng, không trọng số

* Output: Tập hợp các cộng đồng được phát hiện - mỗi cộng đồng được thể hiện là list các node

* Ý nghĩa thực tế: có rất nhiều, ví dụ một số ứng dụng:
  * Tiếp thị: xác định các cụm khách hàng có chung sở thích trong một mạng thể hiện quan hệ giữa người mua và sản phẩm trên sàn thương mại điện tử để xây dựng hệ thống tư vấn hiệu quả
  * Kết bạn, mở rộng mối quan hệ
  * ...


Ứng dụng này cần phải được tăng tốc do vì khi recommend 1 sản phẩm cho khách hàng, ta cần thời gian nhanh để đem lại trải nghiệm tốt nhất cho khách hàng. Trong khi đó thuật toán Girvan Newman có thời gian thực thi khá chậm nên việc song song hóa là rất cần thiết.

## 2. Tiền xử lí dữ liệu

### 1. Tải dữ liệu

In [ ]:
!wget https://raw.githubusercontent.com/mansiganatra/Girvan-Newman-Implementation-using-Spark/master/ub_sample_data.csv

--2022-04-25 05:48:32--  https://raw.githubusercontent.com/mansiganatra/Girvan-Newman-Implementation-using-Spark/master/ub_sample_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1777828 (1.7M) [text/plain]
Saving to: ‘ub_sample_data.csv’

ub_sample_data.csv  100%[===================>]   1.70M  --.-KB/s    in 0.07s   

2022-04-25 05:48:33 (25.1 MB/s) - ‘ub_sample_data.csv’ saved [1777828/1777828]



In [ ]:
import csv

In [ ]:
fileData = []
header = []
with open('ub_sample_data.csv', newline ='') as csvfile:
    file = csv.reader(csvfile, delimiter=',')
    for row in file:
        if not header:
            header.append(row)
            continue
        fileData.append(row)
    csvfile.close()

### 2. Tạo graph từ input

#### Tạo graph

In [ ]:
filterThreshold = 7
setUsers = {}

for [user, item] in fileData:
    if user not in setUsers:
        setUsers[user] = {item}
    else:
        setUsers[user].add(item)

gPyObject = {}

# filter and make adjacency list of graph
for user1, item1 in setUsers.items():
    for user2, item2 in setUsers.items():
        if user1 == user2: continue
        if(len(item1 & item2) >= filterThreshold):
            if user1 not in gPyObject:
                gPyObject[user1] = [user2]
            else:
                gPyObject[user1].append(user2)
cnt = 0
for key, val in gPyObject.items():
    print(key, val)
    cnt += 1
    if cnt == 5:
        break

#### Thay đổi định dạng user từ string sang integer

In [ ]:
transferUser = {}
cnt = 0
for key in gPyObject.keys():
    transferUser[key] = cnt
    cnt += 1
g = [[] for _ in range(cnt)]
for key, val in gPyObject.items():
    user1 = transferUser[key]
    for it in val:
        user2 = transferUser[it]
        g[user1].append(user2)
g[:5]

#### Ghi graph ra file

In [ ]:
with open('graph.txt', "w+") as fp:
    fp.write(str(cnt))
    fp.write("\n")
    for i in g:
        line = " ".join([str(item) for item in i])
        fp.write(line)
        fp.write("\n")

    fp.close()

## 3. Cài đặt tuần tự

### 1. Thiết kế

Các bước thực hiện:
1. Xác định độ đo betweenness cho các cạnh trong đồ thị bằng thuật toán Girvan Newman
2. Loại bỏ cạnh có betweenness lớn nhất khỏi graph
3. Tính độ đo modularity để lưu lại cách chia communities tốt nhất
4. Thực hiện lại bước 2 đến khi không còn cạnh nào

https://colab.research.google.com/drive/1R2Gt2wPEeEL-9_kGfXJRUy24zv1APglX?usp=sharing

### 2. Đánh giá

Độ đo modularity:
sử dụng công thức trên networkx để đánh giá cách chia communities \
Kiểm tra độ chính xác: 
Kết quả chia communities tốt nhất với kết quả chạy bằng thư viện networkx
Thời gian chạy:
Thực hiện 2 version: \
    + ver1: code python bình thường và thêm decorator @jit \
    + ver2: chuyển code về sử dụng mảng numpy và sử dụng decorator @jit(nopython=True)

# Mục mới